<a href="https://colab.research.google.com/github/BrunoStrufaldi/Automa-o-GoogleColab/blob/main/Automa%C3%A7%C3%A3oTitulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala as bibliotecas necessárias
!pip install openpyxl
!pip install unidecode

import pandas as pd
import re
from unidecode import unidecode

def extrair_informacoes(nome, descricao):
    """
    Extrai informações chave do nome e da descrição do produto.
    """
    info = {}
    nome = str(nome)
    descricao = str(descricao)
    info['modelo'] = ''

    # 1. Extrair Marca
    marcas = ['Atlas', 'Electrolux', 'Mueller', 'Esmaltec', 'Realce', 'Clarice', 'Dako']
    info['marca'] = ''
    for marca in marcas:
        if re.search(r'\b' + marca + r'\b', nome, re.IGNORECASE):
            info['marca'] = marca
            break
    if not info.get('marca'):
        for marca in marcas:
            if re.search(r'\b' + marca + r'\b', descricao, re.IGNORECASE):
                info['marca'] = marca
                break

    # 2. Extrair Peças e garantir a ordem
    pecas = []
    if 'queimador' in nome.lower() or 'queimadores' in nome.lower(): pecas.append('Queimadores')
    if 'espalhador' in nome.lower() or 'espalhadores' in nome.lower(): pecas.append('Espalhadores')
    if 'grelha' in nome.lower() or 'grelhas' in nome.lower(): pecas.append('Grelhas')
    info['pecas'] = sorted(list(set(pecas)), key=['Queimadores', 'Espalhadores', 'Grelhas'].index)

    # 3. Extrair Número de Bocas
    bocas_match = re.search(r'(\d)\s?(bcs|bc|bocas|b\b)', nome, re.IGNORECASE)
    if bocas_match:
        info['bocas'] = f"{bocas_match.group(1)} Bocas"

    # 4. Extrair Modelo
    modelo_str = nome
    noise_words = [
        'KIT', 'JOGO', 'PEÇAS', 'GRELHA', 'GRELHAS', 'ESMALTADA', 'ESMALTADO',
        'ESPALHADOR', 'ESPALHADORES', 'QUEIMADOR', 'QUEIMADORES',
        'PARA', 'FOGÃO', 'FOGÕES', 'ORIGINAL'
    ]
    if info.get('marca'):
        modelo_str = re.sub(r'\b' + info['marca'] + r'\b', '', modelo_str, flags=re.IGNORECASE)
    modelo_str = re.sub(r'\b(\d[GTPMC].*?)\b', '', modelo_str, flags=re.IGNORECASE)
    bocas_pattern_match = re.search(r'(\d+\s?(?:bcs|bc|bocas|b\b))', modelo_str, re.IGNORECASE)
    if bocas_pattern_match:
        modelo_str = modelo_str.replace(bocas_pattern_match.group(1), '')
    for word in noise_words:
        modelo_str = re.sub(r'\b' + word + r'\b', '', modelo_str, flags=re.IGNORECASE)
    modelo_str = re.sub(r'[^\w\s.]', '', modelo_str)
    modelo_limpo = ' '.join(modelo_str.split())
    if modelo_limpo:
        if info.get('marca') and info['marca'].lower() == 'electrolux':
            info['modelo'] = modelo_limpo.upper()
        else:
            info['modelo'] = modelo_limpo.title()

    # 5. Extrair configuração das bocas
    config = {}
    for line in descricao.split('\n'):
        line = line.strip()
        match = re.search(r'^(\d+)\s*-', line)
        if not match: continue
        count = match.group(1)
        line_upper = line.upper()
        if 'TRIPLA CHAMA' in line_upper: config['TC'] = f"{count}TC"
        elif 'GIGANTE' in line_upper: config['GT'] = f"{count}GT"
        elif 'GRANDES' in line_upper or 'GRANDE' in line_upper: config['G'] = f"{count}G"
        elif 'MÉDIOS' in line_upper or 'MÉDIO' in line_upper: config['M'] = f"{count}M"
        elif 'PEQUENOS' in line_upper or 'PEQUENO' in line_upper: config['P'] = f"{count}P"
    config_list = []
    order = ['TC', 'GT', 'G', 'M', 'P']
    for key in order:
        if key in config: config_list.append(config[key])
    info['config_bocas'] = " ".join(config_list)

    # 6. Extrair lista de compatibilidade
    info['compatibilidade'] = ''
    compat_match = re.search(r'compat[ií]ve(?:l|is)\s+com(.*?)(?:OBS:|ANTES DA COMPRA|$)', descricao, re.IGNORECASE | re.DOTALL)
    if compat_match:
        lista_modelos = compat_match.group(1)
        lista_modelos = re.sub(r'^\s*(?:os modelos|os fogões \d bocas)\s*[:\-]?\s*', '', lista_modelos, flags=re.IGNORECASE)
        cleaned_models = ' '.join(lista_modelos.split()).strip()
        if cleaned_models:
            info['compatibilidade'] = cleaned_models

    # 7. Keywords Adicionais
    info['sabaf'] = 'Sabaf' if 'sabaf' in descricao.lower() else ''
    return info

def corte_inteligente(texto, limite):
    if len(texto) <= limite: return texto
    texto_cortado = texto[:limite]
    ultimo_espaco = texto_cortado.rfind(' ')
    return texto_cortado[:ultimo_espaco] if ultimo_espaco != -1 else texto_cortado

def criar_titulo_ml(info):
    pecas_plural = info.get('pecas', [])
    pecas_plural_str_plus = ' + '.join(pecas_plural) if pecas_plural else "Peças"
    pecas_plural_str_noplus = ' '.join(pecas_plural) if pecas_plural else "Peças"
    pecas_singular_list = [p[:-2] if p.endswith('es') else (p[:-1] if p.endswith('s') else p) for p in pecas_plural]
    pecas_singular_str_noplus = ' '.join(pecas_singular_list) if pecas_singular_list else "Peça"

    marca = info.get('marca', '')
    modelo = info.get('modelo', '')
    bocas = info.get('bocas', '')
    config_bocas = info.get('config_bocas', '')
    bocas_num = "".join(filter(str.isdigit, bocas))
    bocas_full = f"{bocas_num} Bocas" if bocas_num else ""
    bocas_abbr = f"{bocas_num}B" if bocas_num else ""
    base_produto = f"Fogão {marca} {modelo}"

    def check(title):
        cleaned = ' '.join(title.split())
        return cleaned if len(cleaned) <= 59 else None

    templates = [
        f"Kit {pecas_plural_str_plus} {base_produto} {bocas_full} {config_bocas}",
        f"Kit {pecas_plural_str_plus} {base_produto} {bocas_full}",
        f"{pecas_plural_str_plus} {base_produto} {bocas_full}",
        f"Kit {pecas_plural_str_plus} {base_produto} {bocas_abbr} {config_bocas}",
        f"Kit {pecas_plural_str_plus} {base_produto} {bocas_abbr}",
        f"{pecas_plural_str_plus} {base_produto} {bocas_abbr}",
        f"Kit {pecas_plural_str_noplus} {base_produto} {bocas_abbr}",
        f"{pecas_plural_str_noplus} {base_produto} {bocas_abbr}",
        f"{pecas_singular_str_noplus} {base_produto} {bocas_abbr}",
    ]

    for t in templates:
        titulo_final = check(t)
        if titulo_final:
            return titulo_final

    fallback_title = f"{pecas_singular_str_noplus} {base_produto} {bocas_abbr}"
    return corte_inteligente(' '.join(fallback_title.split()), 59)

# --- FUNÇÃO SHOPEE REFEITA COM NOVA LÓGICA DE 100 CARACTERES ---
def criar_titulo_shopee(info, index):
    # 1. Preparar todos os componentes
    pecas_str = ' + '.join(info.get('pecas', []))
    marca = info.get('marca', '')
    modelo = info.get('modelo', '')
    bocas = info.get('bocas', '')
    config_bocas = info.get('config_bocas', '')
    sabaf = info.get('sabaf', '')
    compatibilidade_bruta = info.get('compatibilidade', '')

    prefixo = f"Kit {pecas_str}" if pecas_str else "Kit Peças Reposição"

    # Cria as duas versões de título base
    bocas_abbr = bocas.replace("Bocas", "B")
    base_full = f"{prefixo} Fogão {marca} {modelo} {bocas} {sabaf} {config_bocas}"
    base_abbr = f"{prefixo} Fogão {marca} {modelo} {bocas_abbr} {sabaf} {config_bocas}"

    titulo_limpo_full = ' '.join(base_full.split())
    titulo_limpo_abbr = ' '.join(base_abbr.split())

    # 2. Lógica de remoção de duplicados da compatibilidade
    compatibilidade_limpa = ''
    if compatibilidade_bruta:
        palavras_para_remover = set()
        titulo_normalizado = unidecode(titulo_limpo_full.upper())
        for palavra in titulo_normalizado.split('+'):
            for sub_palavra in palavra.split():
                palavras_para_remover.add(sub_palavra)
        if modelo:
            palavras_para_remover.add(unidecode(modelo.upper()))

        if marca and compatibilidade_bruta.upper().startswith(marca.upper()):
            compatibilidade_bruta = re.sub(r'^' + re.escape(marca) + r'\s*[:\-]?\s*', '', compatibilidade_bruta, flags=re.IGNORECASE).strip()

        modelos_compat = [m.strip() for m in re.split(r',| - ', compatibilidade_bruta) if m.strip()]

        modelos_filtrados = []
        numeros_no_titulo = {p for p in palavras_para_remover if p.isdigit()}

        for m in modelos_compat:
            m_normalizado = unidecode(m.upper())
            if m_normalizado in palavras_para_remover:
                continue

            partes = m.split()
            if len(partes) > 1 and partes[0] in numeros_no_titulo:
                modelos_filtrados.append(" ".join(partes[1:]))
            else:
                modelos_filtrados.append(m)

        compatibilidade_limpa = ", ".join(modelos_filtrados)

    # 3. Construir e testar os títulos
    def check_shopee(title):
        cleaned = ' '.join(title.split())
        return cleaned if len(cleaned) <= 100 else None

    # Cenário 1: Com compatibilidade
    if compatibilidade_limpa:
        modelos = [m.strip() for m in compatibilidade_limpa.split(',') if m.strip()]

        # Tenta com "Bocas" por extenso
        for i in range(min(len(modelos), 4), 0, -1):
            sufixo = " e mais" if len(modelos) > i else ""
            info_adicional = f"Compatível com {', '.join(modelos[:i])}{sufixo}"
            titulo = check_shopee(f"{titulo_limpo_full} 🔥 {info_adicional}")
            if titulo: return titulo

        # Tenta com "B" abreviado
        for i in range(min(len(modelos), 4), 0, -1):
            sufixo = " e mais" if len(modelos) > i else ""
            info_adicional = f"Compatível com {', '.join(modelos[:i])}{sufixo}"
            titulo = check_shopee(f"{titulo_limpo_abbr} 🔥 {info_adicional}")
            if titulo: return titulo

    # Cenário 2: Com benefícios (ou sem nada se não couber)
    beneficio_completo = "Alta Resistência e Encaixe perfeito"
    beneficio_curto = "Alta Resistência"

    templates_beneficio = [
        f"{titulo_limpo_full} 🔥 {beneficio_completo}",
        f"{titulo_limpo_full} 🔥 {beneficio_curto}",
        f"{titulo_limpo_abbr} 🔥 {beneficio_completo}",
        f"{titulo_limpo_abbr} 🔥 {beneficio_curto}",
        f"{titulo_limpo_full} 🔥",
        f"{titulo_limpo_abbr} 🔥"
    ]

    for t in templates_beneficio:
        titulo = check_shopee(t)
        if titulo: return titulo

    # Fallback final, caso o título base seja muito grande
    return corte_inteligente(titulo_limpo_abbr, 100)

# --- BLOCO PRINCIPAL ---
nome_arquivo_entrada = '/Titulo.xlsx'
nome_arquivo_saida = 'titulos_gerados.xlsx'
try:
    if '.xlsx' in nome_arquivo_entrada.lower():
        df = pd.read_excel(nome_arquivo_entrada)
        print(f"Arquivo Excel '{nome_arquivo_entrada}' lido com sucesso!")
    else:
        df = pd.read_csv(nome_arquivo_entrada)
        print(f"Arquivo CSV '{nome_arquivo_entrada}' lido com sucesso!")

    titulos_ml = []
    titulos_shopee = []
    for index, row in df.iterrows():
        informacoes = extrair_informacoes(row['Nome do Produto'], row['Descrição do Produto'])
        titulos_ml.append(criar_titulo_ml(informacoes))
        titulos_shopee.append(criar_titulo_shopee(informacoes, index))
        if (index + 1) % 100 == 0:
            print(f"Processando linha {index+1}/{len(df)}...")
    df_saida = pd.DataFrame({
        'Titulo Mercado Livre (60 caracteres)': titulos_ml,
        'Titulo Shopee (120 caracteres)': titulos_shopee
    })
    df_saida.to_excel(nome_arquivo_saida, index=False)
    print(f"\nProcesso finalizado! O arquivo '{nome_arquivo_saida}' foi gerado com sucesso.")

except FileNotFoundError:
    print(f"ERRO: Arquivo '{nome_arquivo_entrada}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao processar o arquivo: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.4 MB/s eta 0:00:00
Arquivo Excel '/Titulo.xlsx' lido com sucesso!
Processando linha 100/1179...
Processando linha 200/1179...
Processando linha 300/1179...
Processando linha 400/1179...
Processando linha 500/1179...
Processando linha 600/1179...
Processando linha 700/1179...
Processando linha 800/1179...
Processando linha 900/1179...
Processando linha 1000/1179...
Processando linha 1100/1179...

Processo finalizado! O arquivo 'titulos_gerados.xlsx' foi gerado com sucesso.
